In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Análise exploratória de dados

- Limpeza de outliers
- Análise da utilidade de colunas para o modelo final
- Preenchimento de colunas com nulos / Desconsideração de coluna

## Vamos nos livrar dos pontos fora da curva (outliers)

### Como limparemos esses outliers?

### O que concluímos?

## Vamos analisar as colunas individualmente

### Chutes a gol

### Impedimentos

### Escanteios

### Chutes fora

### Faltas

### Cruzamentos

### Laterais

### Chutes bloqueados

### Contra-ataques

### Tiro de meta

### Tratamentos

### Substituições

### Tiros-livres

### Defesas difíceis

### Posse

### Position

## Conclusões da analise de colunas

# Modelagem